In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn import datasets, linear_model
import pandas as pd
from pandas import DataFrame, Series
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import statsmodels.api as stats

import warnings
warnings.filterwarnings("ignore")

# Problem 1

In [2]:
df_nyc= pd.read_csv('NYC_stop_and_frisk.dat', sep='\s+',skiprows=5)
df_nyc.head(4)

,stops,pop,past.arrests,precinct,eth,crime
0,75,1720,191,1,1,1
1,36,1720,57,1,1,2
2,74,1720,599,1,1,3
3,17,1720,133,1,1,4


## Problem 1.2 : What fraction of the total stops correspond to \white/back/hispanic? What fraction of the population corresponds to \white/black/hispanic?

In [3]:
total= df_nyc.sum()[0]
whiteStops = df_nyc[df_nyc['eth']==3].sum()[0]
hispanicStops = df_nyc[df_nyc['eth']==2].sum()[0]
blackStops = df_nyc[df_nyc['eth']==1].sum()[0]
white_s = whiteStops /total
black_s = blackStops /total
hispanic_s = hispanicStops /total
print("The white stops are :"+ str(white_s))
print("The black stops are :"+ str(black_s))
print("The hispanic stops are :"+ str(hispanic_s))

The white stops are :0.12915842337543754
The black stops are :0.5312966063004109
The hispanic stops are :0.3395449703241516


## Problem 1.3: Use a Poisson regression to model the number of stops, controlling for ethnicity and using the number of past arrests as an exposure input.1

In [4]:
eth_dummies = pd.get_dummies(df_nyc['eth'])
df_nyc_new = pd.concat([df_nyc,eth_dummies],axis = 1)
df_nyc_new =df_nyc_new.rename(columns={1:"Black",2:"Hispanic",3:"White","past.arrests": "Past_arrests"})

In [5]:
from patsy import dmatrices

df_nyc_new['Past_arrests'].replace(0,1,inplace=True)
# formula = """stops ~ pop + Black + Hispanic +White"""
formula = """stops ~ pop + Black + Hispanic +White"""

response, predictors = dmatrices(formula, df_nyc_new, return_type='dataframe')
po_results = stats.GLM(response, predictors, family=stats.families.Poisson(), exposure=df_nyc_new['Past_arrests']).fit()
print(po_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  stops   No. Observations:                  900
Model:                            GLM   Df Residuals:                      896
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -94260.
Date:                Thu, 17 Oct 2019   Deviance:                   1.8318e+05
Time:                        23:01:21   Pearson chi2:                 2.79e+05
No. Iterations:                     6   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4953      0.004   -131.435      0.000      -0.503      -0.488
pop         8.991e-07    8.1e-08     11.102      0.0

## Problem 1.4 : According to the output of your model, what fraction fewer or more stops does each ethnicity have with respect to the others, in proportion to arrest rates of the previous year? Note that you can just pick a baseline ethnicity and just compare everything to that.

In [6]:
## baseline white ethinicty
from patsy import dmatrices

df_nyc_new['Past_arrests'].replace(0,1,inplace=True)   
# formula = """stops ~ pop + Black + Hispanic +White""" 
## remove white ethinicity
formula = """stops ~ pop + Black + Hispanic"""

response, predictors = dmatrices(formula, df_nyc_new, return_type='dataframe')
po_results = stats.GLM(response, predictors, family=stats.families.Poisson(), exposure=df_nyc_new['Past_arrests']).fit()
print(po_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  stops   No. Observations:                  900
Model:                            GLM   Df Residuals:                      896
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -94260.
Date:                Thu, 17 Oct 2019   Deviance:                   1.8318e+05
Time:                        23:01:21   Pearson chi2:                 2.79e+05
No. Iterations:                     6   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8002      0.009    -89.279      0.000      -0.818      -0.783
pop         8.991e-07    8.1e-08     11.102      0.0

In [7]:
df = pd.read_html(po_results.summary().tables[1].as_html(),header=0,index_col=0)[0]
black_coeff=df['coef'].values[2]
hispanic_coeff=df['coef'].values[3]

In [8]:
import math

hispanic_expo  = math.exp(hispanic_coeff)
black_expo = math.exp(black_coeff)
black_percentage = (black_expo -1)*100
hispanic_percentage = (hispanic_expo -1)*100

print("Keeping white as baseline:")
print("The black ethnicity has : "+ str(black_percentage) + "% more stops")
print("The hispanic ethnicity has : "+ str(hispanic_percentage)  + "% more stops")

Keeping white as baseline:
The black ethnicity has : 18.874722441765623% more stops
The hispanic ethnicity has : 27.928330097003705% more stops


## Problem 1.5: Next, add the 75 precincts, and again solve the Poisson regression model.

In [9]:
precinct_dummies = pd.get_dummies(df_nyc['precinct'])
df_precinct = pd.concat([df_nyc_new,precinct_dummies],axis = 1)
df_precinct
# df_new = pd.concat([df, df_x], axis=1)
# df_nyc_new =df_nyc_new.rename(columns={1:"Black",2:"Hispanic",3:"White","past.arrests": "Past_arrests"})

# Taking white race as the baseline.

df_precinct['Past_arrests'].replace(0,1,inplace=True)

formula = """stops ~ pop + Black + Hispanic + precinct_dummies"""

response, predictors = dmatrices(formula, df_precinct, return_type='dataframe')
po_results = stats.GLM(response, predictors, family=stats.families.Poisson(), exposure=df_precinct['Past_arrests']).fit()
print(po_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  stops   No. Observations:                  900
Model:                            GLM   Df Residuals:                      822
Model Family:                 Poisson   Df Model:                           77
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -72943.
Date:                Thu, 17 Oct 2019   Deviance:                   1.4055e+05
Time:                        23:01:21   Pearson chi2:                 2.15e+05
No. Iterations:                     7   Covariance Type:             nonrobust
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -1.1267      0.013    -83.858      0.000      -1.153      -1.100
pop                   

## Task 1.6 Now, controlling for precincts, according to your model, what fraction fewer or more stops  does each ethnicity have with respect to the others, in proportion to arrest rates of the  previous year? (Again, just report with respect to a chosen ethnicity as a baseline).

In [10]:
df6 = pd.read_html(po_results.summary().tables[1].as_html(),header=0,index_col=0)[0]
black_coeff=df6['coef'].values[2]
hispanic_coeff=df6['coef'].values[3]
hispanic_coeff

0.565

In [11]:
hispanic_expo  = math.exp(hispanic_coeff)
black_expo = math.exp(black_coeff)
black_percentage = (black_expo -1)*100
hispanic_percentage = (hispanic_expo -1)*100

print("Keeping white as baseline:")
print("The black ethnicity has : "+ str(black_percentage) + "% percentage more stops")
print("The hispanic ethnicity has : "+ str(hispanic_percentage)  + "% percentage more stops")

Keeping white as baseline:
The black ethnicity has : 78.1757078194389% percentage more stops
The hispanic ethnicity has : 75.9447782721815% percentage more stops


# Problem 2

In [12]:
nba = pd.read_csv('nba_cc_fake_data.csv', sep=',')  ## read data

In [13]:
nba.head()
nba2 = nba.copy()

In [14]:
nba = nba2.rename(columns = {"Unnamed: 0":"Id"})  ## use a more readable field name

In [15]:
nba.head()

,Id,Comp,Height,Points,Salary
0,0,9.0,76.0,27.0,0.0
1,1,7.0,78.0,39.0,0.0
2,2,9.0,76.0,39.0,0.0
3,3,9.0,74.0,39.0,0.0
4,4,9.0,74.0,26.0,0.0


In [16]:
X = nba.loc[:,["Comp","Height","Points"]].values
y = nba.loc[:,["Salary"]].values

In [17]:
# split into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=1) 

## 2.1

### Linear regression is not appropriate for this data because the salary, or the variable we are trying to predict, is not available for most of the points in the data. Therefore, there may or may not be any sort of association between the given explanatory variables and the outcome variable. It is thus hard to use linear regression to find a relationship between the two sets of variables.

## 2.2 - Linear Regression

In [18]:
# least squares regression:
OLS = LinearRegression()
OLS.fit(X_train,y_train).predict(X_test)
y_pred_OLS = OLS.fit(X_train, y_train).predict(X_test)
r2_score_OLS = r2_score(y_test, y_pred_OLS)
r2_score_OLS

0.17136027274846133

## 2.3 - Composite Model

In [19]:
# create a column for salary dummy variable: 1 if player has a salary, 0 otherwise
nba["salary_dummy"] = np.where(nba["Salary"] > 0 , 1, 0)

In [20]:
X_LR = nba.loc[:,["Comp", "Height", "Points"]].values
y_LR = nba.loc[:,["salary_dummy"]].values

In [21]:
# 80% of data are training data and 20% are test data
X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split(X_LR, y_LR, test_size = .2, random_state=25)

In [22]:
LogReg = LogisticRegression(fit_intercept = True)
LogReg.fit(X_train_LR,y_train_LR)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
y_predict_LR = LogReg.predict_proba(X_test_LR)[:,1]

In [24]:
auc = roc_auc_score(y_true = y_test_LR, y_score = y_predict_LR)
print("The computed AUC on the test set: " + str(auc))

The computed AUC on the test set: 0.9116361551965034


In [25]:
clf1 = LogisticRegression(fit_intercept = True)
LogReg.fit(X_train_LR, y_train_LR)

clf2 = LinearRegression()  
OLS.fit(X_train, y_train)
lr = LogisticRegression()
sclf = StackingClassifier(classifiers = [clf1, clf2], meta_classifier = lr)

In [38]:
sclf.fit(X_train_LR,y_train_LR)
y_predict = sclf.predict_proba(X_test_LR)[:,1]

In [39]:
r2_score_sclf = r2_score(y_test_LR, y_predict)
print("The R-squared score for the composite model: " + str(r2_score_sclf))

The R-squared score for the composite model: 0.33297111106003296


## 2.4 - Predict Salary

In [28]:
# inputs
a = np.array([9,78,46]) # comp: 9, 78 inches, 46 points

In [29]:
X_test_ex = a.reshape(-1,3) # get a 1 row by 3 columns array

In [30]:
y_predict = LogReg.predict_proba(X_test_ex)  # LR model predicting whether or not player gets in NBA
y_predict

array([[0.93780953, 0.06219047]])

In [31]:
print("The probability of this player making it to the NBA is: " + str(y_predict[0][1])) 

The probability of this player making it to the NBA is: 0.06219047131859633


In [32]:
expected_salary = OLS.predict(X_test_ex) 
expected_salary = expected_salary*y_predict[0][1]
expected_salary

array([[6542.515653]])

### According to our model, the predicted salary for a high school basketball player who is 6' 6" tall, is averaging 46 points per game, and is playing in the second most competitive league is \$6542.52